In [8]:
! apt update -y
! apt install wget unzip
! pip install torchvision torch


Hit:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
Ign:2 https://mirrors.tencent.com/debian stretch InRelease                     
Get:3 https://mirrors.tencent.com/debian stretch-updates InRelease [93.6 kB]   
Err:3 https://mirrors.tencent.com/debian stretch-updates InRelease             
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY 04EE7237B7D453EC NO_PUBKEY 648ACFD622F3D138
Get:4 https://mirrors.tencent.com/debian stretch Release [118 kB]              
Get:5 https://mirrors.tencent.com/debian stretch Release.gpg [3,177 B]         
Ign:5 https://mirrors.tencent.com/debian stretch Release.gpg                   
Hit:6 https://deb.nodesource.com/node_16.x bionic InRelease                
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease          
Reading package lists... Done
W: GPG error: https://mirrors.tencent.com/debian stretch-updates InRelease: The following signatures couldn't be ve

In [9]:
! wget https://docker-76009.sz.gfp.tencent-cloud.com/github/cube-studio/inference/resnet50.pth

--2022-07-29 16:29:48--  https://docker-76009.sz.gfp.tencent-cloud.com/github/cube-studio/inference/resnet50.pth
Resolving docker-76009.sz.gfp.tencent-cloud.com (docker-76009.sz.gfp.tencent-cloud.com)... 14.22.9.36, 183.47.104.100, 183.47.100.48, ...
Connecting to docker-76009.sz.gfp.tencent-cloud.com (docker-76009.sz.gfp.tencent-cloud.com)|14.22.9.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102530333 (98M) [application/octet-stream]
Saving to: ‘resnet50.pth’

resnet50.pth        100%[===================>]  97.78M  5.96MB/s    in 15s     

2022-07-29 16:30:03 (6.44 MB/s) - ‘resnet50.pth’ saved [102530333/102530333]



In [12]:
from torchvision import models
# from torchsummary import summary
import torch
from torchvision.models.resnet import resnet50
device = torch.device('cpu')

model = resnet50(pretrained=False, progress=True)   # 创建模型
model.load_state_dict(torch.load("resnet50.pth",map_location='cpu')) # 加载模型参数

# 查看模型结构
print(model)

# 查看模型参数
for name, parameters in model.named_parameters():
    print(name, ':', parameters.size())


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [13]:

model.eval()   # 转为推理模式
example_input = torch.rand(1,3,224,224)
traced_model = torch.jit.trace(model, example_input)
traced_model.save('./resnet50_all.pt')

# model=torch.load('resnet50_all.pt')  # 加载全部模型


In [17]:
# 将pt模型转为服务包
! pip install torch-model-archiver
! mkdir 20220801
! torch-model-archiver --model-name resnet50 --version 1.0 --handler image_classifier --serialized-file resnet50_all.pt --export-path ./20220801 -f

# http 输入输出 https://pytorch.org/serve/default_handlers.html


Looking in indexes: https://mirrors.tencent.com/pypi/simple/, https://mirrors.tencent.com/repository/pypi/tencent_pypi/simple
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [5]:
# 看看压缩后文件
! mv resnet50.mar resnet50.zip
! unzip resnet50.zip

/bin/bash: unzip: command not found


In [ ]:
! pip install requests
# 客户端请求服务端
# # pytorch
import requests
import base64
SERVER_URL = 'http://xx.xx.xx.xx:8080/predictions/resnet50'
# IMAGE_PATH = 'smallcat.jpg'
# files = {'data': open(IMAGE_PATH, 'rb')}
image_base64 ='...'
files={'data':base64.b64decode(image_base64)}
response = requests.post(SERVER_URL, files=files)
print(response.json())
print(response.content)
print(response.status_code)